In [49]:
import sys
sys.path.append('../')
import joblib
import requests
import streamlit as st
import tensorflow as tf
from types import SimpleNamespace
import wikipedia 
import openai
import gradio as gr
import torch
from transformers import pipeline
from transformers import AutoTokenizer
from web.model import load_model
from web.prediction import run_prediction
from tools import ggsearch as gg
from tools.ggsearch import GoogleSearch,find_most_similar_documents

In [44]:
question = "Đâu là thủ đô của Việt Nam?"
search = GoogleSearch()
urls,documents = search.gg_search(question)
    
most_similar_documents = find_most_similar_documents(question,documents)
context = most_similar_documents[0]    

['https://thanthienmoitruong.com/tin-tuc.html', 'https://vi.wikipedia.org/wiki/H%C3%A0_N%E1%BB%99i', 'https://vinpearl.com/vi/thu-do-ha-noi', 'https://quynhluu2.edu.vn/Giao-vien/DANH-SACH-CAC-KINH-DO-THU-DO-CUA-VIET-NAM-765.html', 'https://thuvienphapluat.vn/phap-luat/ha-noi-tro-thanh-thu-do-viet-nam-tu-khi-nao-thu-do-ha-noi-co-phai-la-trung-tam-chinh-tri-hanh-chinh-920987-122537.html', 'https://danguykhoicqvadn.thuathienhue.gov.vn/?gd=14&cn=66&tc=1448', 'https://thanthienmoitruong.com/tin-tuc/27/ha-nam-nguon-nuoc-o-nhiem-nghiem-trong.html', 'https://vi.wikipedia.org/wiki/Th%E1%BB%A7_%C4%91%C3%B4_Vi%E1%BB%87t_Nam', 'https://thuvienphapluat.vn/phap-luat/thu-do-nuoc-cong-hoa-xa-hoi-chu-nghia-viet-nam-duoc-quy-dinh-the-nao-bieu-tuong-cua-thu-do-la-hinh--282574-81065.html', 'https://dangcongsan.vn/xay-dung-dang/ha-noi-xung-tam-mot-thu-do-van-minh-hien-dai-niem-tin-yeu-cua-ca-nuoc-593534.html']


In [45]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

In [50]:
model = load_model('../model/qa_large_model/')

In [52]:
question, answer = prediction.run_prediction(model, tokenizer, context, question)

- Question: Đâu là thủ đô của Việt Nam?
- Answer: thành phố Hà Nội
